### Multi step model (simple encoder-decoder)

In this notebook, we demonstrate how to:
- prepare time series data for training a RNN forecasting model
- get data in the required shape for the keras API
- implement a RNN model in keras to predict the next 3 steps ahead (time *t+1* to *t+3*) in the time series. This model uses a simple encoder decoder approach in which the final hidden state of the encoder is replicated across each time step of the decoder. 
- enable early stopping to reduce the likelihood of model overfitting
- evaluate the model on a test dataset

The data in this example is taken from the GEFCom2014 forecasting competition<sup>1</sup>. It consists of 3 years of hourly electricity load and temperature values between 2012 and 2014. The task is to forecast future values of electricity load.

<sup>1</sup>Tao Hong, Pierre Pinson, Shu Fan, Hamidreza Zareipour, Alberto Troccoli and Rob J. Hyndman, "Probabilistic energy forecasting: Global Energy Forecasting Competition 2014 and beyond", International Journal of Forecasting, vol.32, no.3, pp 896-913, July-September, 2016.

In [112]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from collections import UserDict
from IPython.display import Image
%matplotlib inline

from common.utils import load_data, mape, TimeSeriesTensor, create_evaluation_df

pd.options.display.float_format = '{:,.12f}'.format
np.set_printoptions(precision=12)
warnings.filterwarnings("ignore")

In [113]:
Paras = {
    'M0':['M0','Cic'],
    'e':['OMEGA','i0','omega','I_dot','Cus','Crs','e'],
    'sqrt_A':['Cuc','Crc','Del_n','Crs','sqrt_A','OMEGA_dot','Cus'],
    'OMEGA':['OMEGA','e','i0','omega'],
    'i0':['e','i0','omega','OMEGA'],
    'omega':['omega','e','OMEGA','i0'],
    'I_dot':['I_dot','e','Crs','Cuc'],
    'Cic':['M0','Cic'],
    'Cis':['Cis'],
    'OMEGA_dot':['OMEGA_dot','sqrt_A','Crc','Del_n','Cus'],
    'Cuc':['Cuc','e','sqrt_A','I_dot','Crs'],
    'Cus':['Cus','sqrt_A','OMEGA_dot','Crc','Del_n','Cus'],
    'Crc':['Crc','sqrt_A','OMEGA_dot','Cus','Del_n'],
    'Crs':['Crs','e','sqrt_A','I_dot','Cuc'],
    'Del_n':['Crc','sqrt_A','OMEGA_dot','Cus','Del_n'],
}


# enter variable

In [114]:
var_name = 'sqrt_A'

In [115]:
df = pd.read_csv("hourinterpol_2.csv")
a = pd.to_datetime(df['Epoch_Time_of_Clock'])
#df = df.drop(Paras['sqrt_A'],axis =1 )
df = df.loc[:,Paras[var_name]]
df = df.set_index(a)
df.head(40)
df2 = df[18:162]
df2

,Cuc,Crc,Del_n,Crs,sqrt_A,OMEGA_dot,Cus
Epoch_Time_of_Clock,,,,,,,
2017-11-02 00:00:00,0.000002170679,282.345850295000,0.000000005812,36.353610284600,"5,153.602763320000",-0.000000009775,0.000005108000
2017-11-02 01:00:00,0.000001993000,284.079285802000,0.000000005701,34.631191221200,"5,153.603491160000",-0.000000009578,0.000004941882
2017-11-02 02:00:00,0.000001786201,286.244425213000,0.000000005574,32.617279215000,"5,153.604207810000",-0.000000009341,0.000004734210
2017-11-02 03:00:00,0.000001551876,288.817689488000,0.000000005429,30.346223094400,"5,153.604885810000",-0.000000009065,0.000004485336
2017-11-02 04:00:00,0.000001296614,291.750981479000,0.000000005269,27.902367914500,"5,153.605504350000",-0.000000008754,0.000004198000
2017-11-02 05:00:00,0.000001139468,294.432287517000,0.000000005123,26.519971949500,"5,153.606196300000",-0.000000008462,0.000003930207
2017-11-02 06:00:00,0.000001298718,295.758236959000,0.000000005000,28.447548750700,"5,153.607225440000",-0.000000008293,0.000003789842
2017-11-02 07:00:00,0.000001692416,296.269810698000,0.000000005000,32.863475436100,"5,153.608338260000",-0.000000008247,0.000003761573
2017-11-02 08:00:00,0.000001948073,298.042242561000,0.000000005000,35.976382395300,"5,153.608777650000",-0.000000008219,0.000003711935


In [71]:
df.columns

Index(['Cuc', 'Crc', 'Del_n', 'Crs', 'sqrt_A', 'OMEGA_dot', 'Cus'], dtype='object')

In [8]:
#df = df.iloc[1110:,:]


In [116]:
train = df2
valid = df.iloc[162:210]
test = df.iloc[210:258]
#test.tail()
#test.head()
valid
#valid.shape

,Cuc,Crc,Del_n,Crs,sqrt_A,OMEGA_dot,Cus
Epoch_Time_of_Clock,,,,,,,
2017-11-08 00:00:00,-0.000004078697,182.920908673000,0.000000004682,-81.323138002400,"5,153.637249970000",-0.000000009077,0.000010699754
2017-11-08 01:00:00,-0.000004241766,176.780896665000,0.000000004630,-83.848545460800,"5,153.635166700000",-0.000000008869,0.000010865458
2017-11-08 02:00:00,-0.000004395862,169.358963486000,0.000000004575,-86.275912050500,"5,153.632522960000",-0.000000008626,0.000011061412
2017-11-08 03:00:00,-0.000004540923,160.607698154000,0.000000004517,-88.641518774400,"5,153.629309070000",-0.000000008348,0.000011288635
2017-11-08 04:00:00,-0.000004674295,150.536680752000,0.000000004458,-90.935382100100,"5,153.625576730000",-0.000000008000,0.000011547428
2017-11-08 05:00:00,-0.000004733643,140.466285759000,0.000000004394,-92.083434185900,"5,153.622790020000",-0.000000007753,0.000011821670
2017-11-08 06:00:00,-0.000004599740,133.060928383000,0.000000004318,-89.971033614000,"5,153.623622640000",-0.000000007605,0.000012073410
2017-11-08 07:00:00,-0.000004214885,131.806680426000,0.000000004270,-82.978021299400,"5,153.626146540000",-0.000000007593,0.000012139773
2017-11-08 08:00:00,-0.000003593564,140.662965090000,0.000000004334,-70.248842819000,"5,153.624081100002",-0.000000007615,0.000011750485


# enter parameters and Satellite PRN

In [117]:
var_name = 'sqrt_A'
sat_var = 2

In [96]:
#df = df.iloc[5 : , :]

In [97]:

df

,Cuc,Crc,Del_n,Crs,sqrt_A,OMEGA_dot,Cus
Epoch_Time_of_Clock,,,,,,,
2017-11-01 06:00:00,-0.000000056746,302.833250809000,0.000000004790,-4.294667901400,"5,153.612089970000",-0.000000008114,0.000003320075
2017-11-01 07:00:00,0.000000178852,303.605223792000,0.000000005000,1.822865561500,"5,153.610610930000",-0.000000008244,0.000003357716
2017-11-01 08:00:00,0.000000320525,308.871338708000,0.000000005000,5.172251626000,"5,153.609370740000",-0.000000008233,0.000003150879
2017-11-01 09:00:00,0.000000468219,313.955811691000,0.000000005053,7.750722588900,"5,153.608101500000",-0.000000008229,0.000002954205
2017-11-01 10:00:00,0.000000717635,314.363034984000,0.000000005150,11.482858663000,"5,153.606549150000",-0.000000008375,0.000003012465
2017-11-01 11:00:00,0.000001066823,309.741451318000,0.000000005321,16.622242129800,"5,153.604777400000",-0.000000008678,0.000003343457
2017-11-01 12:00:00,0.000001418383,303.800298739000,0.000000005503,21.783216017400,"5,153.603158950000",-0.000000009000,0.000003742573
2017-11-01 13:00:00,0.000001721231,298.514122391000,0.000000005659,26.237920873000,"5,153.601877710000",-0.000000009299,0.000004100454
2017-11-01 14:00:00,0.000001974840,293.942598410000,0.000000005789,29.988946846400,"5,153.600915030000",-0.000000009548,0.000004412859


# Enter number of entries per day

In [ ]:
'''entry = 6
print(df.shape[0])
no_of_entries = df.shape[0]//entry
valid = (no_of_entries * 70)//100
test = (no_of_entries * 85)//100
indexes = df.index
#print(valid , test , indexes)
valid_start_dt = indexes[int(valid)*int(entry)] 
test_start_dt = indexes [int(test)*int(entry)] 
test_start_dt = str(test_start_dt)
valid_start_dt = str(valid_start_dt)
print(test_start_dt,valid_start_dt)
print(type(test_start_dt))
valid_start_dt = '2017-11-07 00:00:00'
test_start_dt = '2017-11-09 00:00:00'''


Load data into Pandas dataframe

# enter lag and no. of outputs

In [118]:
"""total = len(df)
t = total*70/100
t = round(t)
indexes = df.index
valid_start_dt = str(indexes[t])
t = total*85/100
t = round(t)
test_start_dt = str(indexes[t])
print(valid_start_dt , test_start_dt)
"""
T = 24
HORIZON = 24

Create training set containing only the model features

In [ ]:
train = df.copy()[df.index < valid_start_dt][Paras['sqrt_A']]
train.head()

In [ ]:
train.shape

Scale data to be in range (0, 1). This transformation should be calibrated on the training set only. This is to prevent information from the validation or test sets leaking into the training data.

# Enter variable to be predicted

In [119]:
from sklearn.preprocessing import StandardScaler
import pickle



y_scalar = StandardScaler()
y_scalar.fit(train[[var_name]])


X_scaler = StandardScaler()
train[Paras['sqrt_A']] = X_scaler.fit_transform(train)

from sklearn.externals import joblib

new_scaler = StandardScaler()

joblib.dump(new_scaler , 'my_scaler.pkl')     # save to disk

loaded_scaler = joblib.load('my_scaler.pkl')



Use the TimeSeriesTensor convenience class to:
1. Shift the values of the time series to create a Pandas dataframe containing all the data for a single training example
2. Discard any samples with missing values
3. Transform this Pandas dataframe into a numpy array of shape (samples, time steps, features) for input into Keras

The class takes the following parameters:

- **dataset**: original time series
- **H**: the forecast horizon
- **tensor_structure**: a dictionary discribing the tensor structure in the form { 'tensor_name' : (range(max_backward_shift, max_forward_shift), [feature, feature, ...] ) }
- **freq**: time series frequency
- **drop_incomplete**: (Boolean) whether to drop incomplete samples

In [120]:
tensor_structure = {'X':(range(-T+1, 1), Paras['sqrt_A'])}
train_inputs = TimeSeriesTensor(train, var_name, HORIZON, {'X':(range(-T+1, 1), Paras['sqrt_A'])} ,freq = None)

In [121]:
train_inputs.dataframe

tensor                       target                                  \
feature                           y                                   
time step                       t+1             t+2             t+3   
Epoch_Time_of_Clock                                                   
2017-11-02 23:00:00 -1.062183482068 -0.995000852850 -0.933099506151   
2017-11-03 00:00:00 -0.995000852850 -0.933099506151 -0.879757609641   
2017-11-03 01:00:00 -0.933099506151 -0.879757609641 -0.837690307986   
2017-11-03 02:00:00 -0.879757609641 -0.837690307986 -0.796674547437   
2017-11-03 03:00:00 -0.837690307986 -0.796674547437 -0.734687379946   
2017-11-03 04:00:00 -0.796674547437 -0.734687379946 -0.655211622520   
2017-11-03 05:00:00 -0.734687379946 -0.655211622520 -0.588010131521   
2017-11-03 06:00:00 -0.655211622520 -0.588010131521 -0.568724962482   
2017-11-03 07:00:00 -0.588010131521 -0.568724962482 -0.635338910838   
2017-11-03 08:00:00 -0.568724962482 -0.635338910838 -0.770705726827   
2017-11-03 09:00:00 -0.635338910838 -0.770705726827 -0.903991151614   
2017-11-03 10:00:00 -0.770705726827 -0.903991151614 -0.999847372698   
2017-11-03 11:00:00 -0.903991151614 -0.999847372698 -1.060821665721   
2017-11-03 12:00:00 -0.999847372698 -1.060821665721 -1.091106991783   
2017-11-03 13:00:00 -1.060821665721 -1.091106991783 -1.094897255059   
2017-11-03 14:00:00 -1.091106991783 -1.094897255059 -1.076386359809   
2017-11-03 15:00:00 -1.094897255059 -1.076386359809 -1.039768210122   
2017-11-03 16:00:00 -1.076386359809 -1.039768210122 -0.989235767097   
2017-11-03 17:00:00 -1.039768210122 -0.989235767097 -0.928982934995   
2017-11-03 18:00:00 -0.989235767097 -0.928982934995 -0.863204561067   
2017-11-03 19:00:00 -0.928982934995 -0.863204561067 -0.796092663336   
2017-11-03 20:00:00 -0.863204561067 -0.796092663336 -0.731841145889   
2017-11-03 21:00:00 -0.796092663336 -0.731841145889 -0.674644856065   
2017-11-03 22:00:00 -0.731841145889 -0.674644856065 -0.628696754963   
2017-11-03 23:00:00 -0.674644856065 -0.628696754963 -0.598190746670   
2017-11-04 00:00:00 -0.628696754963 -0.598190746670 -0.587320735533   
2017-11-04 01:00:00 -0.598190746670 -0.587320735533 -0.599277181938   
2017-11-04 02:00:00 -0.587320735533 -0.599277181938 -0.614209065209   
2017-11-04 03:00:00 -0.599277181938 -0.614209065209 -0.591033665789   
2017-11-04 04:00:00 -0.614209065209 -0.591033665789 -0.529489748741   
...                             ...             ...             ...   
2017-11-05 18:00:00  0.784388662775  0.869594611705  0.931109293134   
2017-11-05 19:00:00  0.869594611705  0.931109293134  0.964401177835   
2017-11-05 20:00:00  0.931109293134  0.964401177835  0.964938737012   
2017-11-05 21:00:00  0.964401177835  0.964938737012  0.928189498362   
2017-11-05 22:00:00  0.964938737012  0.928189498362  0.849621932916   
2017-11-05 23:00:00  0.928189498362  0.849621932916  0.724704511621   
2017-11-06 00:00:00  0.849621932916  0.724704511621  0.548905705508   
2017-11-06 01:00:00  0.724704511621  0.548905705508  0.321463500863   
2017-11-06 02:00:00  0.548905705508  0.321463500863  0.128341144949   
2017-11-06 03:00:00  0.321463500863  0.128341144949  0.131644775849   
2017-11-06 04:00:00  0.128341144949  0.131644775849  0.253921555842   
2017-11-06 05:00:00  0.131644775849  0.253921555842  0.188891049311   
2017-11-06 06:00:00  0.253921555842  0.188891049311 -0.046674899604   
2017-11-06 07:00:00  0.188891049311 -0.046674899604 -0.113879219832   
2017-11-06 08:00:00 -0.046674899604 -0.113879219832  0.076097943784   
2017-11-06 09:00:00 -0.113879219832  0.076097943784  0.351325381254   
2017-11-06 10:00:00  0.076097943784  0.351325381254  0.616566479416   
2017-11-06 11:00:00  0.351325381254  0.616566479416  0.864775441409   
2017-11-06 12:00:00  0.616566479416  0.864775441409  1.092742944983   
2017-11-06 13:00:00  0.864775441409  1.092742944983  1.297260611048   
2017-11-06 14:00:00  1.092742944983  1.297260611048  1.475117231201   
2017-11-06 15:00:00  1.2

In [102]:
train_inputs['target'].shape

(97, 24)

Construct validation set (keeping T hours from the training set in order to construct initial features)

In [123]:
#look_back_dt = dt.datetime.strptime(valid_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
#valid = df.copy()[(df.index >=look_back_dt) & (df.index < test_start_dt)][['Crs', 'Del_n', 'Cuc', 'sqrt_A', 'Crc']]
valid[Paras['sqrt_A']] = X_scaler.transform(valid)
print(valid[Paras['sqrt_A']])
valid_inputs = TimeSeriesTensor(valid, var_name, HORIZON, tensor_structure,freq = None)
valid_inputs.dataframe


                                        Cuc             Crc  \
Epoch_Time_of_Clock                                           
2017-11-08 00:00:00 -1,165,778.764231041307 -4.758734244130   
2017-11-08 01:00:00 -1,200,921.100587265333 -4.761782709562   
2017-11-08 02:00:00 -1,234,129.460902974941 -4.765467638306   
2017-11-08 03:00:00 -1,265,390.930469847983 -4.769812569523   
2017-11-08 04:00:00 -1,294,133.401198378997 -4.774812746783   
2017-11-08 05:00:00 -1,306,923.317447328474 -4.779812615021   
2017-11-08 06:00:00 -1,278,066.477615277749 -4.783489314015   
2017-11-08 07:00:00 -1,195,127.923969730036 -4.784112037804   
2017-11-08 08:00:00 -1,061,230.022378457477 -4.779714965352   
2017-11-08 09:00:00   -920,090.326742460835 -4.771642375664   
2017-11-08 10:00:00   -852,361.479882204672 -4.764433429600   
2017-11-08 11:00:00   -870,071.168612591922 -4.759742254632   
2017-11-08 12:00:00   -916,172.480764356791 -4.756224857050   
2017-11-08 13:00:00   -960,211.804495683871 -4.75313110

tensor                             target                        \
feature                                 y                         
time step                             t+1                   t+2   
Epoch_Time_of_Clock                                               
2017-11-08 23:00:00 -485,836.899118098663 -485,855.681710527278   

tensor                                                           \
feature                                                           
time step                             t+3                   t+4   
Epoch_Time_of_Clock                                               
2017-11-08 23:00:00 -485,878.091234177176 -485,903.967950746650   

tensor                                                           \
feature                                                           
time step                             t+5                   t+6   
Epoch_Time_of_Clock                                               
2017-11-08 23:00:00 -485,932.669616175932 -485,952.462402200559   

tensor                                                           \
feature                                                           
time step                             t+7                   t+8   
Epoch_Time_of_Clock                                               
2017-11-08 23:00:00 -485,942.235148108215 -485,919.823934570071   

tensor                                                           \
feature                                                           
time step                             t+9                  t+10   
Epoch_Time_of_Clock                                               
2017-11-08 23:00:00 -485,939.758670676209 -485,995.359253345057   

tensor                      ...                             X  \
feature                     ...                           Cus   
time step                   ...                           t-9   
Epoch_Time_of_Clock         ...                                 
2017-11-08 23:00:00         ...          172,220.438501542085   

tensor                                                         \
feature                                                         
time step                            t-8                  t-7   
Epoch_Time_of_Clock                                             
2017-11-08 23:00:00 154,217.048613525141 139,742.166211583506   

tensor                                                         \
feature                                                         
time step                            t-6                  t-5   
Epoch_Time_of_Clock                                             
2017-11-08 23:00:00 128,698.711232026209 120,989.603616323293   

tensor                                                         \
feature                                                         
time step                            t-4                  t-3   
Epoch_Time_of_Clock                                             
2017-11-08 23:00:00 116,517.763302503721 115,186.110228597361   

tensor                                                         \
feature                                                         
time step                            t-2                  t-1   
Epoch_Time_of_Clock                                             
2017-11-08 23:00:00 116,897.564332633134 121,555.045556081430   

tensor                                    
feature                                   
time step                              t  
Epoch_Time_of_Clock                       
2017-11-08 23:00:00 129,061.473833530370  

[1 rows x 192 columns]

## Implement the RNN

We will implement a RNN forecasting model with the following structure:

In [ ]:
#Image('./images/simple_encoder_decoder.png')

In [ ]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Flatten
from keras.callbacks import EarlyStopping ,ModelCheckpoint

In [ ]:
LATENT_DIM = 64
BATCH_SIZE = 32
EPOCHS = 1000

In [ ]:

model = Sequential()
model.add(LSTM(LATENT_DIM, input_shape=(T,5 ) , return_sequences = True))
model.add(LSTM(LATENT_DIM))
model.add(RepeatVector(HORIZON))
model.add(LSTM(LATENT_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.add(Flatten())

In [ ]:
model.compile(optimizer='RMSprop', loss='mse')

In [ ]:
model.summary()

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=500)

In [ ]:
best_val = ModelCheckpoint(str(sat_var) +'_' +  var_name + '_{epoch:02d}.h5', save_best_only=True, mode='min', period=1)

In [ ]:
history = model.fit(train_inputs['X'],
          train_inputs['target'],
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(valid_inputs['X'], valid_inputs['target']),
          callbacks=[earlystop ,best_val],
          verbose=1 , shuffle = False)

In [ ]:
best_epoch = np.argmin(np.array(history.history['val_loss']))+1
model.load_weights(str(sat_var) +'_' +  var_name + '_{:02d}.h5'.format(best_epoch))

In [ ]:
 model.fit(valid_inputs['X'],
          valid_inputs['target'],
          batch_size=BATCH_SIZE,
          epochs=100,
          callbacks=[earlystop ,best_val],
          verbose=1 , shuffle = False)

## Evaluate the model

In [ ]:
look_back_dt = dt.datetime.strptime(test_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
test = df.copy()[test_start_dt:][['Crs', 'Del_n', 'Cuc', 'sqrt_A', 'Crc']]
test[['Crs', 'Del_n', 'Cuc', 'sqrt_A', 'Crc']] = X_scaler.transform(test)
test_inputs = TimeSeriesTensor(test, var_name, HORIZON, tensor_structure,freq =None)
test_inputs.dataframe.head()

In [ ]:
test_inputs.dataframe.head(50)

In [ ]:
test_inputs.dataframe.shape

In [ ]:
predictions = model.predict(test_inputs['X'])

In [ ]:
predictions

In [ ]:
eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scalar)
print(eval_df.head())
print(eval_df.tail())
print(eval_df.shape)


In [ ]:
eval_df['APE'] = (eval_df['prediction'] - eval_df['actual']).abs() / eval_df['actual']
eval_df.groupby('h')['APE'].mean()

# prediction result

In [ ]:
mape(eval_df['prediction'], eval_df['actual'])
from sklearn.metrics import mean_absolute_error
a = mean_absolute_error(eval_df['prediction'], eval_df['actual'])
a

In [ ]:
eval_df['prediction']

Plot actuals vs predictions at each horizon for first week of the test period. As is to be expected, predictions for one step ahead (*t+1*) are more accurate than those for 2 or 3 steps ahead

In [ ]:
plot_df = eval_df[(eval_df.h=='t+1')][['timestamp', 'actual']]
for t in range(1, HORIZON+1):
    plot_df['t+'+str(t)] = eval_df[ (eval_df.h=='t+'+str(t))]['prediction'].values

fig = plt.figure(figsize=(15, 8))
ax = plt.plot(plot_df['timestamp'], plot_df['actual'], color='red', linewidth=4.0)
ax = fig.add_subplot(111)
ax.plot(plot_df['timestamp'], plot_df['t+1'], color='blue', linewidth=4.0, alpha=0.75)
ax.plot(plot_df['timestamp'], plot_df['t+2'], color='blue', linewidth=3.0, alpha=0.5)
ax.plot(plot_df['timestamp'], plot_df['t+3'], color='blue', linewidth=2.0, alpha=0.25)
plt.xlabel('timestamp', fontsize=12)
plt.ylabel('load', fontsize=12)
ax.legend(loc='best')
plt.show()

# take input here

In [ ]:
a = df.iloc[156:162  , :]
a

In [ ]:
new_df = pd.DataFrame()

In [ ]:
columns = ['Crs', 'Del_n', 'Cuc', 'sqrt_A', 'Crc' ] 

In [ ]:
for key , value in enumerate(columns):
    new_df[value] = a[value]

In [ ]:
new_df = new_df.dropna( how = 'any')

In [ ]:
new_df

In [ ]:
# creating index for output
import datetime
date = new_df.index.date[0]
date + datetime.timedelta(1)

In [ ]:
import datetime
date = new_df.index + datetime.timedelta(days =1)

In [ ]:
date

In [ ]:
new_df.index= date

In [ ]:
new_df.columns

In [ ]:
print(new_df)
print(new_df.columns)

In [ ]:
freq = None
idx_tuples = []
drop_incomplete  = True
new_df[['Crs', 'Del_n', 'Cuc', 'sqrt_A', 'Crc']] = X_scaler.transform(new_df)
new_new_df = new_df.copy()
tensor_structure={'X':(range(-T+1, 1), ['Crs', 'Del_n', 'Cuc', 'sqrt_A', 'Crc'])}
for name, structure in tensor_structure.items():
        rng = structure[0]
        dataset_cols = structure[1]
        for col in dataset_cols:
        # do not shift non-sequential 'static' features
            if rng is None:
                new_df['context_'+col] = new_df[col]
                idx_tuples.append((name, col, 'static'))
            else:
                for t in rng:
                    sign = '+' if t > 0 else ''
                    shift = str(t) if t != 0 else ''
                    period = 't'+sign+shift
                    shifted_col = name+'_'+col+'_'+ period
                    new_new_df[shifted_col] = new_new_df[col].shift(t*-1, freq=freq)
                    idx_tuples.append((name, col, period))
        new_new_df = new_new_df.drop(new_df.columns, axis=1)
        idx = pd.MultiIndex.from_tuples(idx_tuples, names=['tensor', 'feature', 'time step'])
        print(new_df.head())
        new_new_df.columns = idx
        if drop_incomplete:
            new_new_df = new_new_df.dropna(how='any')
            
inputs = {}           
for name, structure in tensor_structure.items():
    rng = structure[0]
    cols = structure[1]
    tensor = new_new_df[name][cols].as_matrix()
    if rng is None:
        tensor = tensor.reshape(tensor.shape[0], len(cols))
    else:
        tensor = tensor.reshape(tensor.shape[0], len(cols), len(rng))
        tensor = np.transpose(tensor, axes=[0, 2, 1])
    inputs[name] = tensor

In [ ]:
print(new_new_df)
print(inputs['X'])

In [ ]:
predictions = model.predict(inputs['X'])

In [ ]:
predictions

In [ ]:
print(predictions.shape)


In [ ]:
results = predictions[-1, :]

In [ ]:
results

In [ ]:
res_df = pd.DataFrame(results , columns = [var_name])
res_df

In [ ]:
res_df.index = date
res_df

In [ ]:
res_df[[var_name]]

In [ ]:
res_df[[var_name]] = y_scalar.inverse_transform(res_df[[var_name]]) 

In [ ]:
#res_df[[var_name]] = y_scalar.inverse_transform(res_df[[var_name]]) 
#pd.set_option("display.precision", 8)
#a = pd.Series( dtype = float)
#list_b = []

#for i in range(res_df.shape[0]):
#    list_b.append( "%.20f"%res_df.iloc[i ,0])
    

#print(list_b)
#for i in range(res_df.shape[0]):
#    res_df.iloc[i , 1] = y_scalar.inverse_transform(np.array(res_df.iloc[i ,0]).reshape(1,-1))[0]
#    c[0 , j] = y_scalar.inverse_transform(np.array(res_df.iloc[i ,0]).reshape(1,-1))[0]
#    j +=1





#print (y_scalar.inverse_transform(np.array(list_b).reshape(-1,1)))    


#print (y_scalar.inverse_transform(list_b))    
#a['prediction']
#res_df['inverted'] = 0
#c = np.array()
#a
#j=0

#for i in range(res_df.shape[0]):
    #res_df.iloc[i , 1] = y_scalar.inverse_transform(np.array(res_df.iloc[i ,0]).reshape(1,-1))[0]
    #c[0 , j] = y_scalar.inverse_transform(np.array(res_df.iloc[i ,0]).reshape(1,-1))[0]
    #j +=1

#b = np.array(-1.08850443363189697266).reshape(1,-1)

#y_scalar.inverse_transform(b)

# final generated output 

In [ ]:
# final generated ouput
res_df

In [ ]:
res_df.to_csv('SA1SqrtA.csv')

In [ ]:
df.tail(6)